In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
csv_file_path = "job_details_full.csv"
df = pd.read_csv(csv_file_path)
df

,Field,Experience,Location,Company Size,Salary,Job Requirements,URL
0,Dược phẩm / Y tế / Công nghệ sinh học,2 năm,Hà Nội,25-99 nhân viên,9 - 12 triệu,"Yêu cầu ứng viên\n• Nữ, tốt nghiệp Đại học, Ca...",https://www.topcv.vn/viec-lam/ke-toan-kiem-han...
1,Du lịch,3 năm,Hà Nội,100-499 nhân viên,15 - 18 triệu,Yêu cầu ứng viên\n- Trình độ: tốt nghiệp Đại h...,https://www.topcv.vn/viec-lam/ke-toan-thue-thu...
2,Xây dựng,Dưới 1 năm,Đà Nẵng,100-499 nhân viên,Thoả thuận,Yêu cầu ứng viên\n- Giới tính: Nam.\n- Trình đ...,https://www.topcv.vn/viec-lam/nhan-vien-ke-toa...
3,Bán lẻ - Hàng tiêu dùng - FMCG,3 năm,Hà Nội,25-99 nhân viên,16 - 20 triệu,Yêu cầu ứng viên\n• Tốt nghiệp Đại học chuyên ...,https://www.topcv.vn/viec-lam/ke-toan-tong-hop...
4,Bán lẻ - Hàng tiêu dùng - FMCG,1 năm,Hồ Chí Minh,25-99 nhân viên,10 - 12 triệu,"Yêu cầu ứng viên\n● Giới tính: Nữ, độ tuổi từ ...",https://www.topcv.vn/viec-lam/ke-toan-ban-hang...
...,...,...,...,...,...,...,...
381,Marketing / Truyền thông / Quảng cáo,1 năm,Hà Nội,25-99 nhân viên,15 - 20 triệu,"Yêu cầu ứng viên\nCó tư duy sáng tạo, nhanh nh...",https://www.topcv.vn/viec-lam/quan-tri-seo-ken...
382,Du lịch,2 năm,Hồ Chí Minh,10-24 nhân viên,15 - 30 triệu,Yêu cầu ứng viên\n1. Có 2-3 năm kinh nghiệm là...,https://www.topcv.vn/viec-lam/bien-tap-video/1...
383,Thời trang,4 năm,Hồ Chí Minh,25-99 nhân viên,Thoả thuận,Yêu cầu ứng viên\nTốt nghiệp đại học chuyên ng...,https://www.topcv.vn/viec-lam/giam-doc-marketi...
384,Marketing / Truyền thông / Quảng cáo,Dưới 1 năm,Hà Nội,25-99 nhân viên,Tới 30 triệu,"Yêu cầu ứng viên\nTốt nghiệp Cao đẳng trở lên,...",https://www.topcv.vn/viec-lam/nhan-vien-kinh-d...


In [3]:
df = df.dropna()

In [4]:
def process_experience(exp):
    if pd.isna(exp):
        return 0
    
    # Convert "2 năm" -> 2
    if "năm" in str(exp):
        match = re.search(r'(\d+)', str(exp))
        if match:
            return int(match.group(1))
    
    # Handle "Dưới 1 năm" -> 1
    if "Dưới 1 năm" in str(exp):
        return 1
    
    # Handle "không yêu cầu" or "Không yêu cầu" -> 0
    if "không yêu cầu" in str(exp).lower() or "không" in str(exp).lower():
        return 0
    
    return 0  # Default case

df['Experience_year'] = df['Experience'].apply(process_experience)

C:\Users\hello\AppData\Local\Temp\ipykernel_14484\2388425961.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Experience_year'] = df['Experience'].apply(process_experience)


In [5]:
def process_salary(salary):
    if pd.isna(salary):
        return {"min": "TBD", "max": "TBD"}
    
    salary = str(salary)
    
    # Case: "x - y triệu"
    if " - " in salary and "triệu" in salary:
        parts = salary.split(" - ")
        min_salary = float(parts[0].strip())
        max_salary = float(parts[1].split(" ")[0].strip())
        return {"min": min_salary, "max": max_salary}
    
    # Case: "Thỏa thuận" or "Thoả thuận"
    if "thoả thuận" in salary.lower() or "thoa thuan" in salary.lower():
        return {"min": "TBD", "max": "TBD"}
    
    # Case: "Tới z triệu"
    if "tới" in salary.lower() and "triệu" in salary.lower():
        match = re.search(r'(\d+)', salary)
        if match:
            z = float(match.group(1))
            return {"min": z/2, "max": z}
    
    # Case: "Từ z triệu"
    if "từ" in salary.lower() and "triệu" in salary.lower():
        match = re.search(r'(\d+)', salary)
        if match:
            z = float(match.group(1))
            return {"min": z, "max": z*1.5}  # Assuming max is 1.5x of min
    
    return {"min": "TBD", "max": "TBD"}  # Default case

In [6]:
df['minSalary'] = df['Salary'].apply(lambda x: process_salary(x)["min"])
df['maxSalary'] = df['Salary'].apply(lambda x: process_salary(x)["max"])

C:\Users\hello\AppData\Local\Temp\ipykernel_14484\3597528151.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['minSalary'] = df['Salary'].apply(lambda x: process_salary(x)["min"])
C:\Users\hello\AppData\Local\Temp\ipykernel_14484\3597528151.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['maxSalary'] = df['Salary'].apply(lambda x: process_salary(x)["max"])


In [7]:
def clean_job_requirements(text):
    if pd.isna(text):
        return ""
    
    # Replace bullet points, special characters and new lines with space
    cleaned_text = re.sub(r'[●•\-\*\n\r\t]', ' ', str(text))
    
    # Remove extra spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    
    return cleaned_text

# Create temporary cleaned column
df['Job_Requirements_Cleaned'] = df['Job Requirements'].apply(clean_job_requirements)

# Rename the cleaned column and drop the original
df['Job Requirements'] = df['Job_Requirements_Cleaned']
df = df.drop(columns=['Job_Requirements_Cleaned'])

C:\Users\hello\AppData\Local\Temp\ipykernel_14484\406067780.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Job_Requirements_Cleaned'] = df['Job Requirements'].apply(clean_job_requirements)
C:\Users\hello\AppData\Local\Temp\ipykernel_14484\406067780.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Job Requirements'] = df['Job_Requirements_Cleaned']


In [8]:
df

,Field,Experience,Location,Company Size,Salary,Job Requirements,URL,Experience_year,minSalary,maxSalary
0,Dược phẩm / Y tế / Công nghệ sinh học,2 năm,Hà Nội,25-99 nhân viên,9 - 12 triệu,"Yêu cầu ứng viên Nữ, tốt nghiệp Đại học, Cao đ...",https://www.topcv.vn/viec-lam/ke-toan-kiem-han...,2,9.0,12.0
1,Du lịch,3 năm,Hà Nội,100-499 nhân viên,15 - 18 triệu,"Yêu cầu ứng viên Trình độ: tốt nghiệp Đại học,...",https://www.topcv.vn/viec-lam/ke-toan-thue-thu...,3,15.0,18.0
2,Xây dựng,Dưới 1 năm,Đà Nẵng,100-499 nhân viên,Thoả thuận,Yêu cầu ứng viên Giới tính: Nam. Trình độ chuy...,https://www.topcv.vn/viec-lam/nhan-vien-ke-toa...,1,TBD,TBD
3,Bán lẻ - Hàng tiêu dùng - FMCG,3 năm,Hà Nội,25-99 nhân viên,16 - 20 triệu,Yêu cầu ứng viên Tốt nghiệp Đại học chuyên ngà...,https://www.topcv.vn/viec-lam/ke-toan-tong-hop...,3,16.0,20.0
4,Bán lẻ - Hàng tiêu dùng - FMCG,1 năm,Hồ Chí Minh,25-99 nhân viên,10 - 12 triệu,"Yêu cầu ứng viên Giới tính: Nữ, độ tuổi từ 22 ...",https://www.topcv.vn/viec-lam/ke-toan-ban-hang...,1,10.0,12.0
...,...,...,...,...,...,...,...,...,...,...
381,Marketing / Truyền thông / Quảng cáo,1 năm,Hà Nội,25-99 nhân viên,15 - 20 triệu,"Yêu cầu ứng viên Có tư duy sáng tạo, nhanh nhạ...",https://www.topcv.vn/viec-lam/quan-tri-seo-ken...,1,15.0,20.0
382,Du lịch,2 năm,Hồ Chí Minh,10-24 nhân viên,15 - 30 triệu,Yêu cầu ứng viên 1. Có 2 3 năm kinh nghiệm làm...,https://www.topcv.vn/viec-lam/bien-tap-video/1...,2,15.0,30.0
383,Thời trang,4 năm,Hồ Chí Minh,25-99 nhân viên,Thoả thuận,Yêu cầu ứng viên Tốt nghiệp đại học chuyên ngà...,https://www.topcv.vn/viec-lam/giam-doc-marketi...,4,TBD,TBD
384,Marketing / Truyền thông / Quảng cáo,Dưới 1 năm,Hà Nội,25-99 nhân viên,Tới 30 triệu,"Yêu cầu ứng viên Tốt nghiệp Cao đẳng trở lên, ...",https://www.topcv.vn/viec-lam/nhan-vien-kinh-d...,1,15.0,30.0


In [9]:
df.to_csv(fr"C:\htN\UIT\lastyear\helping\main\data\job_details_full.csv", index=False)